## 신경망 모델 저장

In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from sklearn.preprocessing import OneHotEncoder

# 훈련 데이터를 불러옵니다.
train_data = pd.read_csv("KBO_TRAIN.csv")

# 특성(X)과 타겟(y)으로 나눕니다.
X_train, y_train = train_data.drop('GRADE', axis=1), train_data['GRADE']

# Convert class labels to be in the range from 0 to 9
y_train = y_train - 1

# 훈련 데이터와 검증 데이터로 나눕니다.
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# One-Hot Encoding을 수행하여 팀 이름을 숫자로 변환
encoder = OneHotEncoder()
X_combined = pd.concat([X_train, X_val])  # Combine the training and validation sets
X_encoded = encoder.fit_transform(X_combined[['TEAM']])

# 데이터 전처리를 진행합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_encoded[:len(X_train)].toarray())
X_val_scaled = scaler.transform(X_encoded[len(X_train):].toarray())

# 신경망 모델의 구조를 정의합니다.
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')  # KBO 팀 수에 해당하는 10개의 클래스 (0부터 9)
])

# 모델을 컴파일합니다.
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 모델을 훈련합니다.
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_val_scaled, y_val))

# 검증 세트로 모델을 평가합니다.
val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val)
print("검증 세트 손실:", val_loss)
print("검증 세트 정확도:", val_accuracy)

# 훈련된 모델을 파일로 저장합니다.
# 훈련된 모델을 파일로 저장합니다.
# 훈련된 모델을 TensorFlow's SavedModel format으로 저장합니다.
model.save("path_to_model_directory", save_format='tf')
model.save("path_to_model_directory.h5", save_format='tf')



Epoch 1/100
1/1 [==============================] - 1s 597ms/step - loss: 2.4291 - accuracy: 0.0417 - val_loss: 2.2900 - val_accuracy: 0.1667
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 2.3943 - accuracy: 0.0417 - val_loss: 2.2862 - val_accuracy: 0.1667
Epoch 3/100
1/1 [==============================] - 0s 34ms/step - loss: 2.3614 - accuracy: 0.0417 - val_loss: 2.2836 - val_accuracy: 0.1667
Epoch 4/100
1/1 [==============================] - 0s 46ms/step - loss: 2.3296 - accuracy: 0.1250 - val_loss: 2.2820 - val_accuracy: 0.1667
Epoch 5/100
1/1 [==============================] - 0s 34ms/step - loss: 2.2987 - accuracy: 0.1250 - val_loss: 2.2808 - val_accuracy: 0.1667
Epoch 6/100
1/1 [==============================] - 0s 29ms/step - loss: 2.2692 - accuracy: 0.1250 - val_loss: 2.2797 - val_accuracy: 0.1667
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 2.2406 - accuracy: 0.1250 - val_loss: 2.2793 - val_accuracy: 0.1667
Epoch 8/100
1/1 [==

INFO:tensorflow:Assets written to: path_to_model_directory\assets


## 2차 테스트

In [44]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from sklearn.preprocessing import OneHotEncoder

# 훈련된 모델을 불러옵니다.
model = joblib.load("path_to_model_file.pkl")

# 테스트 데이터를 불러옵니다.
test_data = pd.read_csv("KBO_TEST.csv")

# 특성(X)과 타겟(y)으로 나눕니다. 테스트 데이터에는 'GRADE' 열이 없으므로 예측 결과를 추가할 열로 사용합니다.
X_test = test_data.drop('GRADE', axis=1)
y_test = test_data['GRADE']

# One-Hot Encoding을 수행하여 'TEAM' 열을 숫자로 변환합니다.
encoder = OneHotEncoder()
X_combined = pd.concat([X_train, X_val, X_test])  # 훈련, 검증, 테스트 데이터를 합쳐서 One-Hot Encoding을 적용합니다.
X_encoded = encoder.fit_transform(X_combined[['TEAM']])
X_test_encoded = X_encoded[len(X_train) + len(X_val):]

# 데이터 전처리를 진행합니다. 표준화를 위해 StandardScaler를 사용합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_encoded[:len(X_train)].toarray())
X_val_scaled = scaler.transform(X_encoded[len(X_train):len(X_train) + len(X_val)].toarray())
X_test_scaled = scaler.transform(X_test_encoded.toarray())

# 테스트 데이터에 대해 예측을 수행합니다.
predictions = model.predict(X_test_scaled)

# 예측 결과는 각 클래스(0부터 9)에 대한 확률값입니다. 예측된 클래스를 구합니다.
predicted_classes = predictions.argmax(axis=1)

# 예측된 클래스 레이블을 원래 범위로 변환합니다 (1부터 10).
predicted_classes = predicted_classes + 1

# 예측된 "GRADE" 열을 테스트 데이터 DataFrame에 추가합니다.
test_data['PREDICTED_GRADE'] = predicted_classes

# 예측 결과가 추가된 test_data DataFrame을 출력합니다.
print(test_data)


1/1 [==============================] - 0s 48ms/step
   SEASON TEAM    AVG    OBP    SLG   ERA  WHIP  GRADE  PREDICTED_GRADE
0      23   LG  0.285  0.373  0.394  3.66  1.33      1                3
1      23   NC  0.266  0.346  0.384  3.67  1.34      4                6
2      23  SSG  0.259  0.336  0.396  4.06  1.50      2                1
3      23   두산  0.257  0.336  0.372  3.90  1.36      3                3
4      23  KIA  0.260  0.331  0.367  3.82  1.44      7                9
5      23   키움  0.254  0.328  0.347  3.77  1.33      9                5
6      23   KT  0.264  0.335  0.360  4.38  1.40      6                3
7      23   한화  0.241  0.328  0.344  3.88  1.34      8               10
8      23   삼성  0.252  0.322  0.356  4.57  1.46     10                8
9      23   롯데  0.255  0.327  0.346  4.53  1.52      5                8


In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from sklearn.preprocessing import OneHotEncoder

# 훈련된 모델을 불러옵니다.
model = keras.models.load_model("path_to_model_directory.h5")

# 테스트 데이터를 불러옵니다.
test_data = pd.read_csv("KBO_TEST.csv")

# 특성(X)과 타겟(y)으로 나눕니다. 테스트 데이터에는 'GRADE' 열이 없으므로 예측 결과를 추가할 열로 사용합니다.
X_test = test_data.drop('GRADE', axis=1)
y_test = test_data['GRADE']

# One-Hot Encoding을 수행하여 'TEAM' 열을 숫자로 변환합니다.
encoder = OneHotEncoder()
X_combined = pd.concat([X_train, X_val, X_test])  # 훈련, 검증, 테스트 데이터를 합쳐서 One-Hot Encoding을 적용합니다.
X_encoded = encoder.fit_transform(X_combined[['TEAM']])
X_test_encoded = X_encoded[len(X_train) + len(X_val):]

# 데이터 전처리를 진행합니다. 표준화를 위해 StandardScaler를 사용합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_encoded[:len(X_train)].toarray())
X_val_scaled = scaler.transform(X_encoded[len(X_train):len(X_train) + len(X_val)].toarray())
X_test_scaled = scaler.transform(X_test_encoded.toarray())

# 테스트 데이터에 대해 예측을 수행합니다.
predictions = model.predict(X_test_scaled)

# 예측 결과는 각 클래스(0부터 9)에 대한 확률값입니다. 예측된 클래스를 구합니다.
predicted_classes = predictions.argmax(axis=1)

# 예측된 클래스 레이블을 원래 범위로 변환합니다 (1부터 10).
predicted_classes = predicted_classes + 1

# 예측된 "GRADE" 열을 테스트 데이터 DataFrame에 추가합니다.
test_data['PREDICTED_GRADE'] = predicted_classes

# 각 팀별로 예상 등수와 확률의 Top 3를 뽑아냅니다.
team_predictions = {}
for team, pred in zip(test_data['TEAM'], predictions):
    if team not in team_predictions:
        team_predictions[team] = []
    team_predictions[team].append((np.argmax(pred) + 1, np.max(pred)))

# 각 팀별로 예상 등수와 확률의 Top 3를 구합니다. (grade, prob)
# 각 팀별로 예상 등수와 확률의 Top 3를 구합니다. (grade, prob)
def get_top3(predictions):
    top3 = sorted(predictions, key=lambda x: x[1], reverse=True)[:3]
    if len(top3) < 3:
        top3.extend([(None, None)] * (3 - len(top3)))
    return top3

top3_teams = {team: get_top3(preds) for team, preds in team_predictions.items()}

# Top 3 결과를 출력합니다.
for team, top3 in top3_teams.items():
    print(f"{team} 팀의 예상 등수 및 확률 Top 3:")
    for rank, (grade, prob) in enumerate(top3, 1):
        grade_str = str(grade) if grade else "None"
        prob_str = f"{prob:.4f}" if prob is not None else "None"
        print(f"   {rank}위: {grade_str} (확률: {prob_str})" if grade_str != "None" else f"   {rank}위: {grade_str} (확률: {prob_str})")


1/1 [==============================] - 0s 47ms/step
LG 팀의 예상 등수 및 확률 Top 3:
   1위: 3 (확률: 0.6221)
   2위: None (확률: None)
   3위: None (확률: None)
NC 팀의 예상 등수 및 확률 Top 3:
   1위: 6 (확률: 0.3016)
   2위: None (확률: None)
   3위: None (확률: None)
SSG 팀의 예상 등수 및 확률 Top 3:
   1위: 1 (확률: 0.3220)
   2위: None (확률: None)
   3위: None (확률: None)
두산 팀의 예상 등수 및 확률 Top 3:
   1위: 3 (확률: 0.4724)
   2위: None (확률: None)
   3위: None (확률: None)
KIA 팀의 예상 등수 및 확률 Top 3:
   1위: 5 (확률: 0.4511)
   2위: None (확률: None)
   3위: None (확률: None)
키움 팀의 예상 등수 및 확률 Top 3:
   1위: 5 (확률: 0.9514)
   2위: None (확률: None)
   3위: None (확률: None)
KT 팀의 예상 등수 및 확률 Top 3:
   1위: 3 (확률: 0.2663)
   2위: None (확률: None)
   3위: None (확률: None)
한화 팀의 예상 등수 및 확률 Top 3:
   1위: 10 (확률: 0.9181)
   2위: None (확률: None)
   3위: None (확률: None)
삼성 팀의 예상 등수 및 확률 Top 3:
   1위: 8 (확률: 0.9393)
   2위: None (확률: None)
   3위: None (확률: None)
롯데 팀의 예상 등수 및 확률 Top 3:
   1위: 8 (확률: 0.5527)
   2위: None (확률: None)
   3위: None (확률: None)


In [59]:
X_train.head()


,SEASON,TEAM,AVG,OBP,SLG,ERA,WHIP
28,22,두산,0.255,0.324,0.365,4.46,1.48
24,22,KIA,0.272,0.349,0.398,4.21,1.42
12,21,한화,0.237,0.334,0.341,4.68,1.49
0,20,LG,0.277,0.349,0.428,4.39,1.42
4,20,삼성,0.268,0.338,0.394,4.78,1.47


In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import joblib
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# 훈련된 모델을 불러옵니다.
model = joblib.load("path_to_model_file.h5")

# 테스트 데이터를 불러옵니다.
test_data = pd.read_csv("KBO_TEST.csv")

# 특성(X)과 타겟(y)으로 나눕니다. 테스트 데이터에는 'GRADE' 열이 없으므로 예측 결과를 추가할 열로 사용합니다.
X_test = test_data.drop('GRADE', axis=1)
y_test = test_data['GRADE']

# One-Hot Encoding을 수행하여 'TEAM' 열을 숫자로 변환합니다.
encoder = OneHotEncoder()
X_combined = pd.concat([X_train, X_val, X_test])  # 훈련, 검증, 테스트 데이터를 합쳐서 One-Hot Encoding을 적용합니다.
X_encoded = encoder.fit_transform(X_combined[['TEAM']])
X_test_encoded = X_encoded[len(X_train) + len(X_val):]

# 데이터 전처리를 진행합니다. 표준화를 위해 StandardScaler를 사용합니다.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_encoded[:len(X_train)].toarray())
X_val_scaled = scaler.transform(X_encoded[len(X_train):len(X_train) + len(X_val)].toarray())
X_test_scaled = scaler.transform(X_test_encoded.toarray())

# 테스트 데이터에 대해 예측을 수행합니다.
@tf.function
def predict(X):
    return model.predict(X)

predictions = predict(X_test_scaled)

# 예측 결과는 각 클래스(0부터 9)에 대한 확률값입니다. 예측된 클래스를 구합니다.
predicted_classes = predictions.argmax(axis=1)

# 예측된 클래스 레이블을 원래 범위로 변환합니다 (1부터 10).
predicted_classes = predicted_classes + 1

# 예측된 "GRADE" 열을 테스트 데이터 DataFrame에 추가합니다.
test_data['PREDICTED_GRADE'] = predicted_classes

# 각 팀별로 예상 등수와 확률의 Top 3를 뽑아냅니다.
team_predictions = test_data.groupby('TEAM')['PREDICTED_GRADE'].apply(list).to_dict()

# 각 팀별로 예상 등수와 확률의 Top 3를 구합니다. (grade, prob)
def get_top3(predictions):
    top3 = sorted(predictions, key=lambda x: x[1], reverse=True)[:3]
    if len(top3) < 3:
        top3.extend([(None, None)] * (3 - len(top3)))
    return top3

top3_teams = {team: get_top3(preds) for team, preds in team_predictions.items()}

# Top 3 결과를 출력합니다.
for team, top3 in top3_teams.items():
    print(f"{team} 팀의 예상 등수 및 확률 Top 3:")
    for rank, (grade, prob) in enumerate(top3, 1):
        grade_str = str(grade) if grade else "None"
        prob_str = f"{prob:.4f}" if prob is not None else "None"
        print(f"   {rank}위: {grade_str} (확률: {prob_str})")


KeyError: 72

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

# KBO_TRAIN.csv 파일 로드
train_data = pd.read_csv("KBO_TRAIN.csv")

# 데이터 전처리
X_train = train_data.drop(['SEASON', 'TEAM', 'GRADE'], axis=1)
y_train = train_data['GRADE']

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 머신러닝 모델 학습
model = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

# 테스트 데이터 로드
test_data = pd.read_csv("KBO_TEST.csv")

# 데이터 전처리
X_test = test_data.drop(['SEASON', 'TEAM', 'GRADE'], axis=1)
y_test = test_data['GRADE']

# 스케일링
X_test_scaled = scaler.transform(X_test)

# 테스트 데이터로 예측
y_pred = model.predict(X_test_scaled)

# 예측 결과 출력
print("테스트 데이터 예측 결과:")
print(y_pred)


테스트 데이터 예측 결과:
[ 2  2  2  2  2 10  2  1  5  2  3  4 10  2  2  2  2  2  5  5  4  3  3  3
  2  3 10  2 10  2]


In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tabulate import tabulate
import joblib

# Load the test data
test_data = pd.read_csv("KBO_TEST.csv")

# Preprocess X_test
X_test = test_data.drop('GRADE', axis=1)
X_test = X_test.drop('SEASON', axis=1)  # Drop the SEASON column if it's not needed
X_test = X_test.drop('TEAM', axis=1)  # Drop the TEAM column if it's not needed

# Standard Scaling (if needed)
scaler = StandardScaler()
X_test_scaled = scaler.fit_transform(X_test)

# Load the pre-trained neural network model
model = joblib.load("path_to_model_file.pkl")  # Replace "path_to_model_file.pkl" with your actual model file path

# Make predictions
pred_top3, pred_detail = model.predict(X_test_scaled)

# Print the results using tabulate
print(tabulate(pred_top3, headers='keys', tablefmt='psql'))


FileNotFoundError: [Errno 2] No such file or directory: 'path_to_model_file.pkl'